In [36]:
"""
HealthBot - Safe Hugging Face Implementation (Token Hidden)
Author: Your Name
Description:
A friendly health information chatbot using Hugging Face models.
Token is NOT stored in the code — must be set via environment variable.
"""

import os
import re
from huggingface_hub import InferenceClient

In [43]:
env="""
HUGGINGFACE_TOKEN= enter your token here
"""

In [38]:


# ======================
# TOKEN HANDLING
# ======================
HF_TOKEN = os.getenv("HUGGINGFACE_TOKEN")

if not HF_TOKEN:
    raise ValueError(
        "❌ Hugging Face token not found.\n"
        "Set it before running:\n"
        "In Colab:  %env HUGGINGFACE_TOKEN=hf_xxx\n"
        "In Terminal: export HUGGINGFACE_TOKEN=hf_xxx"
    )

# ======================
# CLIENT INIT
# ======================
client = InferenceClient(token=HF_TOKEN)

# ======================
# SYSTEM PROMPT
# ======================
SYSTEM_MESSAGE = """
You are a helpful and friendly general health information assistant.

GUIDELINES:
1. Do NOT provide specific diagnoses, treatment plans, or personalized medical advice.
2. Do NOT recommend medications or specific therapies.
3. Responses are for general knowledge only — always tell users to consult a doctor for medical concerns.
4. Be clear, friendly, and use widely accepted public health knowledge.
"""

# ======================
# TIER 1: HARD BLOCK
# ======================
TIER1_PATTERNS = [
    r"\btake\s+\d+\s*(?:mg|milligram|g|gram|ml|milliliter|tablet|capsule|pill)s?(?:\s+(?:once|twice|daily|per\s+day))?\b",
    r"\b(?:prescribe|prescription\s+for)\b.*",
    r"\bover[\s-]?the[\s-]?counter\b.*(?:for|to treat|against)\b.*",
    r"\bapply\s+\w+.*\b(?:to|on)\b.*(?:skin|area|wound|rash|affected)\b",
    r"\byou\s+(?:must|are advised to|are recommended to)\b.*",
    r"\b(?:paracetamol|acetaminophen|ibuprofen|amoxicillin|antibiotic|aspirin)\b.*(?:\d+\s*(?:mg|ml|tablet|pill)|for\s+\w+)",
]

# ======================
# TIER 2: SOFT WARN
# ======================
TIER2_PATTERNS = [
    r"\b(?:paracetamol|acetaminophen|ibuprofen|amoxicillin|antibiotic|aspirin)\b",
    r"\byou\s+should\b.*",
    r"\bit\s+is\s+(?:recommended|advised)\b.*",
    r"\bconsult\s+(?:your|a)\s+doctor\b",
]

In [39]:
# ======================
# SAFETY CHECK FUNCTION
# ======================
def check_medical_safety(text):
    for pattern in TIER1_PATTERNS:
        if re.search(pattern, text, re.IGNORECASE):
            return "BLOCK"
    for pattern in TIER2_PATTERNS:
        if re.search(pattern, text, re.IGNORECASE):
            return "WARN"
    return "ALLOW"

In [40]:
# ======================
# FILTER FUNCTION
# ======================
def filter_response(response: str) -> str:
    status = check_medical_safety(response)
    if status == "BLOCK":
        return "⚠️ I can't provide direct medical instructions. Please consult a qualified healthcare professional."
    elif status == "WARN":
        return response + "\n\n💡 Remember: For medical advice, always consult a qualified healthcare provider."
    return response

In [41]:
# ======================
# CHATBOT RESPONSE FUNCTION
# ======================
def get_chatbot_response(user_query: str) -> str:
    try:
        completion = client.chat.completions.create(
            model="mistralai/Mistral-7B-Instruct-v0.2",
            messages=[
                {"role": "system", "content": SYSTEM_MESSAGE},
                {"role": "user", "content": user_query}
            ],
        )
        raw_text = completion.choices[0].message.content.strip()
        return filter_response(raw_text)
    except Exception as e:
        return f"❌ Error: {str(e)}"

In [42]:
# ======================
# MAIN CHAT LOOP
# ======================
if __name__ == "__main__":
    print("💬 HealthBot (Type '/quit' or press Enter to exit)\n")
    while True:
        user_query = input("You: ").strip()
        if not user_query or user_query.lower() in {"/quit", "quit", "exit"}:
            print("👋 Goodbye!")
            break
        bot_response = get_chatbot_response(user_query)
        print(f"Assistant: {bot_response}\n" + "-"*40)

💬 HealthBot (Type '/quit' or press Enter to exit)

You: "What causes a sore throat?"
Assistant: ⚠️ I can't provide direct medical instructions. Please consult a qualified healthcare professional.
----------------------------------------
You: soure throat
Assistant: A sore throat can be caused by various factors such as viruses, bacteria, or environmental irritants. Common symptoms include pain, redness, and swelling in the throat area. To help soothe a sore throat, here are some general suggestions:

1. Stay hydrated: Drink plenty of fluids like water, clear broth, or caffeine-free teas.
2. Gargle with saltwater: Dissolve half a teaspoon of salt in a glass of warm water, then gargle gently.
3. Use a humidifier: Dry air can irritate a sore throat. A humidifier helps keep the air moist.
4. Rest your voice: Avoid yelling, singing, or whispering for extended periods.
5. Eat soft foods: Try eating foods like applesauce, rice pudding, or oatmeal.
6. Avoid throat irritants: Stay away from spi